In [ ]:


#Import Libraries and Load Raw Dato
import numpy as np 
import pandas as pd
import datetime as dt
import scipy.stats as stats
import matplotlib.pyplot as plt
import warnings
import os
from scipy import interpolate
import matplotlib.pyplot as plt 
max_temp = pd.read_csv('maximum_temperature.csv')
min_temp = pd.read_csv('minimum_temperature.csv')
max_temp.head()

#Clean dat of Null Vaues and Show it is quantity
max_temp.isnull().value_counts(),min_temp.isna().value_counts
count = 0
for mx, mn in zip(np.where(max_temp.isnull())[0], np.where(min_temp.isnull())[0]):
    if mx != mn:
        count +=1
print('Number of Null Values: ', count)


#Fram raw Data, Extracts Date (Y-M-D) Maximum and Minimum Temperatures (Tmax, and Tmin) and calculate the Daily Average ( T=(Tmax+Tmin)/2)


def datetime(row):
    return dt.datetime(row.Year, row.Month, row.Day)
max_temp['Date'] = max_temp.apply(datetime,axis=1)
min_temp['Date'] = min_temp.apply(datetime,axis=1)
max_temp.set_index('Date',inplace=True)
min_temp.set_index('Date', inplace=True)

#MDrops column with useless data, renames the columns left, and merges the data frames
drop_cols =[0,1,2,3,4,6,7]
max_temp.drop(max_temp.columns[drop_cols], axis=1, inplace=True)
min_temp.drop(min_temp.columns[drop_cols], axis=1, inplace=True)
max_temp.rename(columns={'Maximum temperature (Degree F)': 'Tmax'}, inplace=True)
max_temp.rename(columns={'Minimum temperature (Degree F)': 'Tmin'}, inplace=True)
temps = max_temp.merge(min_temp, how=' inner',left_on=['Date'],right_on=['Date'])
#Calculates Daily Average droping null values.

def avg_temp(row):
    return (row.Tmax+row.Tmin)/2
temps['T'] = temps.apply(avg_temp, axis=1)
# drop na values here 
temps  = temps.dropna()
#Divides temperature in seasons wih Summer between May and September and Winter between October and April
temps_season = temps.copy(deep=True)
temps_season['month'] = temps_season. index.month
mask = (temps_season['month'] >5) & (temps['month'] <= 10)
temps_season['winter'] = np.where(mask,1,0)
temps_season['summer'] = np.where(temps_season['winter'] != 1,1,0)
temps_season
#Creates timeseries with $T_{\max }$, $T_{m i n}$ and, Daily Average, and plots: All data, Last 14 years, and Histogram
temps[:].plot(figsize=(8,6))
plt.show()
#Timeseries plot  - 14 year
temps[-5000:].plot(figsize=(8,6))
temps.Tmax.hist(bins=60, alpha=0.6, label ='Tmax')
temps.Tmin.hist(bins=60, alpha=0.6, label ='Tmin')
temps['T'].hist(bins=60, alpha=0.8, label ='T')
plt.legend()
plt.show()
#Plots Histograms of summer and winter temperatures
plt.figure(figsizw=(8,6))
temps_season[temps_season['summer'] == 1]['T'].hist(bins=60, alpha=0.8, label='Summer')
temps_season[temps_season['winter'] == 1]['T'].hist(bins=60, alpha=0.8, label='Winter')
plt.legend()
plt.show()
#imports Q-Q plot, ADFuller, Seasonal Decompose, ACF, and P-ACF tools.
from statsmodels.graphics.api import qqplot
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa,ar_model import AutoReg, ar_select_order, AutoResults

#Plots Rolling Mean over Annual Period (Red) and Rolling Variance over Anual period (Blue).
temps.sort_index(inplace=True)
Years=20
temps['T'].rolling(window = 365*Years).mean().plot(figsize=(12,8), color="tab:red", title="Rolling mean over annual periods", label = 'Mean')
temps['T'].rolling(window = 365*Years).var().plot(figsize=(12,8), color="tab:blue", title="Rolling mean and variance over annual periods", label = 'Variance')
plt.legend()
plt.show()

#Apply decompose Seasonal to time series and plot, Data, Irend, Seasonal, and Residual 
decompose_result = seasonal_decompose(temps['T'], model ='additive', period=int(365), extrapolate_trend='freq')
trend = decompose_result.trend
seasonal = decompose_result.seasonal
residual = decompose_result.resid
#Visualise all Data, Trend, Seasonal, and Residuals
decompose_result.plot()
plt.show()
#Visualise x-Years
Years = 20
years_examine = 365*Years
fig, axs = plt.subplots(3, figsize=(8,6))
fig.suptitle('Removed Trend and Seasonality')
axs[0].plot(trend[~years_examine:])
axs[1].plot(seasonal[~years_examine:])
axs[1].set_ylim([-10,10])

#Apply fitting models and calculate parameters with Residual Sum of Squares(RSS)
#Model 1 fit = a+bt+asin(wt+Lambda) here y_pred = a+bt+a1sin((omega)x +b1)
#Model 2 fit = a+bt+asin(wt+Lambda)+Lambdacos(omega*t +phi) here y_pred=a+bt+a1sin((omega)x+theta)+b1cos((omega)x+phi)

from scipy.stats import norm
from scipy.optimize import check_fit
temp_t = temps['T'].copy(deep=True)
#Define Models
# Model 1 (Sine)
def model_fit(x, a b, a1, b1):
     omega = 2 np.pi/365.25
     y_pred = a +b*x + a1*np.cos(omega*x) + b1*np.sin(omega*x)
     return y_pred
def RSS(y, y_pred):
    return np.sqrt( (y - y_pred)**2 ).sum()
#Model 2(Generalized)
def model_fit_general(x, a, b, a1, b1, theta, phi):
    omega = 2*np.pi/365.25
    y_pred = a + b*x + a1*np.cos(omega*x + theta) + b1*np.sin(omega*x + phi)
return y_pred

if isinstance(temp_t.index, pd.DatetimeIndex):
    first_ord = temp_t.index.map(dt.datetime.toordinal)[0]
    temp_t.index=temp_t.index.map(dt.datetime.toordinal)

params, cov = curve_fit(model_fit, xdata = temp_t.index-first_ord, ydata = temp_t['T'], method='lm')
param_list = ['a', 'b', 'a1', 'b1']
print('\n       Model 1 Parameters \\n')
std_dev = np.sqrt(np.diag(cov))
for name, p, sd in zip( param_list, params, std_dev):
    print('{0} : {1:0.3} CI ~normally [{2:0.2e}, {3:0.2e}]'.format(name, p, p-1.96*sd, p+1.96*sd))
temp_t['Model 1 (Sine)'] = model_fit(temp_t.index-first_ord, *params)
if isinstance(temp_t.index, pd.DatetimeIndex):
    temp_t.index=temp_t.index.map(dt.datetime,toordinal)
params1, cov1 = curve_fit(model_fit_general, xdata = temp_t.index-first_ord, ydata = temp_t['T'], method='lm')
param_list = ['a', 'b', 'a1', 'b1', 'theta', 'phi']
print('\n Model2 Paramerers \n')
std_dev = np.sqrt(np.diag(cov1))
for name, p, sd in zip( param_list, params1, std_dev):
    print(' {0} : {1:0.3} CI ~normally [{2:0.2e}, {3:0.2e}]'.format(name, p, p-1.96*sd, p+1.96*sd))
temp_t['Model 2(Gen)'] = model_fit_general(temp_t.index-first_ord, *params1)
if not isinstance(temp_t.index, pd.DatetimeIndex):
    temp_t.index=temp_t.index.map(dt.datetime.fromordinal)
temp_t[:2000].plot(figsize=(12,4), style=['s', '^-', 'k--'], markersize=4, linewidth=2 )
temp_t[-2000:].plot(fugsize=(12,4), style=['s', '^-', 'k--'], markersize=4, linewidth=2  )   
RSS(temp_t['T'], tem_t['Model 2 (Gen)'])
print('\n Residual Sum of Squares (RSS) \n')
print(' RSS Model 1 (Sine):', round (RSS(temp_t['T'], temp_t['Model 1 (Sine)']),2))
print('  RSS Model 2 (Gen):', round(RSS(temp_t['T'], temp_t['Model 2 (Gen)']),2))

#Calculate parameters with data and fit curve
temp_t = temps['T'].copy(deep=True)
temp_t = temp_t.to_frame()
def model(x, params):
    a,b,a1,b1 = params
    omega = 2*np.pi/365.25
    theta = np.arctan(a1/b1)
    alpha = np.sqrt( a1**2 + b1**2)
    print('Parameters:\n     a{0:0.3}\n   b{1:0.3}\n alpha {2:0.3}\n theta {3:0.3}'.format(a,b,alpha,theta))
    y_pred = a + b*x + alpha*np.sin(omega*x + theta)
    return y_pred

def model_fit(x, a, b, a1, b1):
        omega = 2*np.pi/365.25
        y_pred = a + b*x + a1*np.cos(omega*x) + b1*np.sin(omega*x)
        return y_pred
if isinstance(temp_t.index , pd.DatetimeIndex):
        first_ord = temp_t.index.map(dt.datetime.toordinal)[0]
        temp_t.index=temp_t.index.map(dt.datetime.toordinal)
params_all, cov = curve_fit(model_fit, xdata = temp_t.index-first_ord, ydata = temp_t['T'], method='lm' )

temp_t['Model'] = model(temp_t.index-first_ord, params_all)
if not isinstance(temp_t.index , pd.DatetimeIndex):
     temp_t.index=temp_t.index.map(dt.datetime.fromordinal)
temp_t[-2000:].plot(fifsize=(12,4), style=['s', '^-', 'k-'], markersize=4, linewidth=3 )
plt.show()

#Autocorrelation was applied and Residuals were analysed
#Visualize Residuals after detrending and Removing seasonality from the Daily Average Temperature (DAT) 12 years
if not isinstance(temp_t.index , pd.DatetimeIndex):
     temp_t.index=temp_t.index.map(dt.datetime.fromordinal)
temp_t['res'] = temp_t['T']-temp_t['Model']
temp_t['res'][-5000:].plot(figsize=(12,6))
plt.show()
#visualizes autocorrelation for all the time series (Tlags) and the Last few years(PLags)
Tlags = 30
Plags = 30
fig, axs = plt.subplots(2,2, figsize=(12,8))
fig.suptitle('Residuals after de-trending and removing seasonality from the DAT')
axs[0,0].plot(temp_t['res'])
axs[1,0].plot(temp_t['res'][-2000:])
plot_acf(temp_t['res'], lags = TLags, ax=axs[0,1])
plot_pacf(temp_t['res'], lags = Plags, ax=axs[1,1])
plt.show()
#Probablility distribution was Calculated and plotted
stats.probplot(temp_t['res'], dist ="norm", plot=plt)
plt.title("Normal Probability Plot")
plt.show()
#Plot probability distribution of temperature errors and compares it to normal distribution.
mu, std =norm.fit(temp_t['res'])
z = (temp_t['res'] - mu)/std
plt.hist(temp_t['res'], density=True, alpha=0.6, bins=100, label='Temp Error')
xmin, xmax = plt.xlim()
ymin, ymax = plt.ylim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
data = np.random.randn(100000)
plt.plot(x, p, 'k', linewidth=2, label='Normal Dist')
plt.plot([std*2,std*2], [0,ymax])
print('P(Z > 2): {:0.3}% vs. Normal Distribution: {:0.3}% '.format(len(z[z >= 2])/len(z)*100, (1-norm.cdf(2))*100))
print('Skew    : {:0.3}'.format(stats.skew(z)))
print('Kurtosis    : {:0.3}'.format(stats.kurtosis(z)+3))
plt.ylabel('Probability Density')
plt.xlabel('Temperature Errors')
plt.legend()
plt.show()

#Visualization of Fitted Data
#plots Partial Data and Fit Model
rows=2000
temp_t [['T', 'Model']][-rows:].plot(figsize=(12,6), style=['s', 'k-'], markersize=4, linewidth=3 )
plt.show()
#Autoregression was applied to Temperature Residuals
#Performs as Autoregression of the Residuals and print parameters
residuals = temp_t['res']
resiaduals.index = pd.DatetimeIndex(residuals.index).to_period('D')
model = AutoReg(residuals, lags=1, old_names=True, trends='n')
model_fit = model.fit()
coef = model_fit.params
res = model_fit.resid
res.index = res.index.to_timestamp()
print(model_fit.summary())

#The monthly volatility of the DAT was estimated and Plot
#Estimate volatility based on quadratic variation of temperature process. Plot Monthly Volatility of DAT
temp_t['Day'] = temp_t.index.dayofyear
temp_t['month'] = temp_t.index.month
temp_t['year'] = temp_t.index.year
vol = temp_t.groupby(['year', 'month'])['T'].agg(['mean', 'std'])
vol = vol.reset_index()
vol['std'].plot(figsize=(8,6))
plt.plot([0, len(vol)], [vol['std'].mean(),vol['std'].mean()], 'k', linewidth=2)
plt.ylabel('Std Dev (deg F)')
plt.tittle('Monthly Volatility of Observed Daily Average Temperature', color='k')
print('Trend or long Term volatility is easy: ~', round(vol['std'].mean(),3))
plt.show()
# The dailay volatity of DATA was estimated and Plotted
#Estimate Daily Volatilty of DAT, and Plot
vol =  temp_t.groupby(['Day'])['T'].agg(['mean', 'std'])
vol['std'].plot(color='tab:blue', figsize=(8,6))
plt.ylabel('Std Dev (deg F)', color= 'tab:blue')
plt.xlim(0,364)
plt.show()

#Plot Spline Fit of Volatility
from scipy import interpolate
x= np.array(vol['std'].index)
y = np.array(vol['std'].index)
knot_numbers = 5
x_new = np.linspace(0, 1, knot_numbers+2)[1:-1]
q_knots = np.quantile(x, x_new)
t,c,k = interpolate.splrep(x, y, t=q_knots, s=1)
yfit = interpolate.BSpline(t,c,k)(x)
plt.figure(figsize=(8,4))
plt.plot(x, y, label='Volatility')
plt.plot(x, yfit, label='Spline fit')
plt.ylabel('Std Dev (deg C)')
plt.xlabel('Day in Year')
plt.xlim(0,364)
plt.legend(loc='lower right')
plt.show()
def spline(knots, x, y):
     x_new = np.linspace(0, 1, knots+2)[1:-1]
     t, c, k = interpolate.splrep(x, y, t=np.quantile(x, x_new), s=3)
     yfit = interpolate.BSpline(t,c, k)(x)
     return yfit

knots = [3, 10, 20,30, 50, 80]
i = 0
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(8, 5))
for row in range(2):
     for col in range(3):
          ax[row][col].plot(x, y, '.',c='tab:orange', markersize =4)
          yfit = spline(knots[i], x ,y)
          rss = np.sum( np.square(y-yfit) )
          ax[row][col].plot(x, yfit, 'k', linewidth=2)
          ax[row][col].set_title("Knots Pi" + str(knots[i])+ "\nRSS: " +str(round(rss,2)), color='k')
          ax[row][col].set_xlim(0,366)
          ax[row][col].grid()
          i=i+1
plt.tight_layout()
plt.show()
#estimate Volatility from the quadratic Variation of the Temperature Process
temp_vol = temps['T'].copy(deep=True)
temp_vol = temp_vol.to_frame()
temp_vol['Day'] = temp_vol.index.dayofyear
temp_vol['month'] = temp_vol.index.month
vol = temp_vol.groupby(['Day'])['T'].agg(['mean', 'std'])
days = np.array(vol['std'].index)
T_std = np.array(vol['std'].values)
#fits STd DEV to 5-knots spline
def spline(knots, x, y):
     x_new = np.linspace(0, 1, knots+2)[1:-1]
     t, c, k = interpolate.splrep(x, y, t=np.quantile(x, x_new), s=3)
     yfit = interpolate.BSpline(t,c, k)(x)
     return yfit
volatility = spline(5, days, T_std)
plt.figure(figsize=(8,4))
plt.plot(days, T_std, marker='.', label='Volatility')
plt.plot(days, volatility, linewidth=4, label='Spline fit')
plt.ylabel('Std Dev (deg F)')
plt.xlabel('Day in Year')
plt.xlim(0,364)
plt.legend(loc='Lower right')
plt.show()

#Estimate volatility of the volatility process by using the quadratic variation of sigma
print('Gamma is : ', round(vol['std'].std(),3))
model = AutoReg(vol['std'], lags=1, old_names=Truem trend='n')
model_fit = model.fit()
coef = model_fit.params
res = model_fit.resid
print('Rate of mean reversion of volatility process is    : ', coef[0])
print(model_fit.summary())
#####
#####
#####
#####
#Apply Euler approximation to Time Series
def euler_step(row, kappa, M):
    #*** Function for Euler sheme approximation step in
    #midified OH Dynamics for temperature simulations
    #inputs:
    #dataframe row with columns: T, Tbar, dTbar and vol
    #kappa: rate of mean reversion from data processing code
    #Output:
    # temp:simulated next day temperature
    if row['Tbar_shift'] != np.nan:
        T_i = row['Tbar']
    else:
        T_i = row['Tbar_shift']
    T_det = T_i + row['dTbar']
    T_mrev = kappa*(row['Tbar'] - T_i)
    sigma = row['vol']*np.random.randn(M)
    #riskn = lamda*row['vol']
    return T_det + T_mrev + sigma

#Apply Monte Carlo approximation to time series
def monte_carlo_temp(trading_dates, Tbar_params, vol_model, first_ord, M=1, kappa=0.438):
    #Monte Carlo simulation of temperature
    #Inputs:
    #trading_dates: pandas DatetimeIndex from start to end dates
    #M:number of simulations
    #Tbar_params:parameters used for Tbar model
    #vol_model: fitted volatility model with days in year index
    #first_ord: first ordinal of fitted Tbar model
    # Output:
    # mc_temps: DataFrame of all components and simulated temperatures
    kappa=0.438
    if isinstance(trading_dates, pd.DatetimeIndex):
        trading_date=trading_dates.map(dt.datetime.toordinal)

    #Use Modified Ornstein-Uhlenbeck process with estimated parameters to stimulate Tbar DAT
    Tbars = T_model(trading_date-first_ord, *Tbar_params)
    #Use derivative of modified OH process SDE to calculate change of Tbar 
    dTbars = dT_model(trading_date-first_ord, *Tbar_params)
    

    mc_temps = pd.DataFrame(data=np.array([Tbars, dTbars]).T, index=trading_dates, columns=['Tbar', 'dTbar'])
    mc_temps['day'] = mc_temps.index.dayofyear
    mc_temps['vol'] =vol_model[mc_temps['day']-1]

    mc_temps['T'] =mc_temps['Tbar'].shift(1)
    data = mc_temps.apply(euler_step, args=[kappa, M], axis=1)
    mc_sims = pd.DataFrame(data=[x for x in [y for y in data.values]], index=trading_dates, columns=range(1, M+1))
    return mc_temps, mc_sims

#Ornstein -Uhlenbeck process with Modified OU Stochastic Diff Eq. and parameters a, b alpha,theta.
if isinstance(temp_t.index , pd.DatetimeIndex):
     first_ord = temp_t.index.map(dt.datetime.toordinal)[0]
     temp_t.index=temp_t.index.map(dt.datetime.toordinal)
#Define T
def dT_model(x, a, b, alpha, theta):
     omega=2*np.pi/365.25
     dT = b + alpha*omega*np.cos(omega*x + theta)
     return dT
#input parametes and plot model 
Tbar_params = [62.1, 6.08e-05, 9.03, 1.26]
temp_t['model_fit'] = T_model(temp_t.index-first_ord, *Tbar_params)
if not isinstance(temp_t.index , pd.DatetimeIndex):
    temp_t.index=temp_t.index.map(dt.datetime.fromordinal)

# define trading date range
no_sims = 5
trading_dates = pd.date_range(start='2024-04-22', end='2025-04-22', freq='D')
mc_temps, mc_sims = monte_carlo_temp(trading_dates, Tbar_params, volatility, first_ord, no_sims)

mc_temps

plt.figure(figsize=(12,6))
mc_sims[1].plot(alpha=0.6, linewidth=2, marker='.')
mc_temps["Tbar"].plot(linewidth=4)
plt.legend(loc='lower right')
plt.show()

no_sims = 10000
trading_dates_winter = pd.date_range(start='2023-07-01', end='2023-07-01', freq='D')
mc_temps_winter, mc_sims_winter = monte_carlo_temp(trading_dates_winter, Tbar_params, volatility, first_ord, no_sims)
trading_dates_summer = pd.date_range(start='2023-01-01', end='2023-01-01', freq='D')
mc_temps_summer, mc_sims_summer = monte_carlo_temp(trading_dates_summmer, Tbar_params, volatility, first_ord, no_sims)
plt.fugure(figsize=(12,6))
plt.title('Winter vs Summer Temperature MC Sims')
Tbar_summer = mc_temps_summer.iloc[-1,:]['Tbar']
Tbar_winter = mc_temps_summer.iloc[-1,:]['Tbar']
plt.hist(mc_sins_summer.iloc[-1,:],bins=20, alpha=0.5, label='Summer', color='tab:orange')
plt.plot([Tbar_summer, Tbar_summer],[0,1600], linewidth=4, lable='Tbar_summer',  color='tab:orange')
plt.title('Summer vs Winter Temperature MC Sims')
plt.hist(mc_sims_winter.iloc[-1,:],bins=20, alpha=0.5, label='Winter', color='tab:blue')
plt.plot([Tbar_winter, Tbar_winter],[0,1600], linewidth=4, lable='Tbar_winter',  color='tab:blue')
plt.legend()
plt.show()








     
              




















          











FileNotFoundError: [Errno 2] No such file or directory: 'maximum_temperature.csv'